# Import software libraries

In [ ]:
import sys                                                    # Read system parameters.
import numpy as np                                            # Work with multi-dimensional arrays.
import pandas as pd
import spacy                                                  # Process text.
import nltk                                                   # Process text.
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import re                                                     # Work with regular expressions.
import warnings                                               # Suppress warnings.
warnings.filterwarnings('ignore')

# Summarize software libraries used.
print('Libraries used in this project:')
print('- Python {}'.format(sys.version))
print('- NumPy {}'.format(np.__version__))
print('- pandas {}'.format(pd.__version__))
print('- spaCy {}'.format(spacy.__version__))
print('- NLTK {}'.format(nltk.__version__))

# Read and preview the text data

In [ ]:
complaints_data = pd.read_csv('/home/student/CDSP/Text/data/consumer_loan_complaints.csv')

complaints_data.head()

# Check the shape of the data

In [ ]:
complaints_data.shape

# Retrieve information about the data

In [ ]:
complaints_data.info()

In [ ]:
complaints_data.Issue.value_counts()

In [ ]:
complaints_data['Company response to consumer'].value_counts()

# Extract a subset of data to consider only consumer complaints

In [ ]:
print('Number of users with no complaints data:',
      complaints_data['Consumer complaint narrative'].isnull().sum())

In [ ]:
# Remove records with missing complaint narratives.

text_data = complaints_data[~complaints_data \
                            ['Consumer complaint narrative'].isnull()] \
                           [['user_id', 'Consumer complaint narrative']]

text_data.head(n = 3)

In [ ]:
text_data.shape

# Preview an example of the consumer complaints

In [ ]:
sample_text = text_data['Consumer complaint narrative'].iloc[0]
sample_text

# Tokenize the sample text into sentences

In [ ]:
nlp = spacy.load('/home/student/spacy_data/' +
                 'en_core_web_sm/en_core_web_sm-3.7.1/')

document = nlp(sample_text)

In [ ]:
for sentence in document.sents:
    print(sentence)

# Tokenize the sentences into words

In [ ]:
sentence = nlp('It is showing a debt of {$250.00} per month along ' \
               'with XXXX different amounts charged off of the ' \
               '{$950.00} ( plus interest ) and another {$5100.00} ' \
               'that INCLUDES the {$950.00}.')

In [ ]:
for token in sentence:
    print(token.text)

# Identify the parts of speech for each token

In [ ]:
pos = []

for token in sentence:
    pos.append({'Word': token,
                'Part of Speech': token.pos_
               })

pd.DataFrame(pos)

# Identify stop words

In [ ]:
stop = []

for token in sentence:
    stop.append({'Word': token,
                 'Stop Word?': token.is_stop
               })

pd.DataFrame(stop)

# Stem the text

In [ ]:
text = 'This was a car loan but the company is providing ' \
       'fraudulent information this is damaging my credit!'

print(word_tokenize(text))

In [ ]:
stemmer = SnowballStemmer(language = 'english')

for token in word_tokenize(text):
    print(token, '-->', stemmer.stem(token))

# Lemmatize the text

In [ ]:
parsed_text = nlp(text)

for token in parsed_text:
    print(token, '-->', token.lemma_)

# Transform the text

In [ ]:
def spacy_cleaner(original_text):
    """Cleans text data to be processed.
    Removes punctuation, whitespace, numbers, stopwords from the text
    and lemmatizes each token."""

    nlp = spacy.load('/home/student/spacy_data/' +
                     'en_core_web_sm/en_core_web_sm-3.7.1/')

    final_tokens = []
    parsed_text = nlp(original_text)

    for token in parsed_text:
        if token.is_punct or token.is_space or token.like_num or token.is_stop:
            pass
        else:
            if token.lemma_ == '-PRON-':
                final_tokens.append(str(token))  # Keep pronouns as they are.
            else:
                sc_removed = re.sub('[^a-zA-Z]', '', str(token.lemma_))
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)
    joined = ' '.join(final_tokens)
    preprocessed_text = re.sub(r'(.)\1+', r'\1\1', joined)

    return preprocessed_text

In [ ]:
# Apply transformation to sample.

spacy_cleaner(sample_text)

In [ ]:
# Compare to sample before transformation.

sample_text

In [ ]:
# Apply transformation to entire dataset.

text_data['consumer_complaints_cleaned'] = \
text_data['Consumer complaint narrative'].apply(lambda x: spacy_cleaner(x))

text_data.head(n = 3)